# [蝦皮 Shopee](https://shopee.tw/)  (json 資料)
## [參考影片](https://www.youtube.com/watch?v=jV6eHoLzD2E)

### 導入套件

In [1]:
import time
import requests # 讀取網頁資料
import bs4
import json
from selenium import webdriver

### 搜尋 "Text" 網址會轉為`https://shopee.tw/search?keyword="text"`

In [2]:
text = 'cabinzero'
url = f'https://shopee.tw/search?keyword={text}'

headers = {'user-agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'}
r = requests.get(url, headers=headers)
soup = bs4.BeautifulSoup(r.text, "html.parser")

In [3]:
for k in r.headers:
    print(k , r.headers[k])

Server SGW
Date Mon, 13 Jan 2020 05:24:41 GMT
Content-Type text/html
Transfer-Encoding chunked
Connection keep-alive
Last-Modified Fri, 10 Jan 2020 10:09:54 GMT
Vary Accept-Encoding
ETag W/"5e184d72-2156"
Content-Encoding gzip
Expires Mon, 13 Jan 2020 05:24:40 GMT
Cache-Control no-cache, no-cache, no-store
Content-Security-Policy frame-ancestors 'self' *.shopeemobile.com *.shopee.tw *.shopee.cn *.facebook.com; 


### Chrome F12 觀察, 並搜尋body 可以知道商品內容並不在其中

In [4]:
print(soup.select('body')[0].prettify())

<body>
 <noscript>
  <iframe height="0" src="https://www.googletagmanager.com/ns.html?id=GTM-WJZQSJF" style="display:none;visibility:hidden" width="0">
  </iframe>
 </noscript>
 <div id="main">
 </div>
 <div id="modal">
 </div>
 <script src="https://deo.shopeemobile.com/shopee/shopee-pcmall-live-sg/assets/modules~bundle.a1765fa60dc6d58fd61b.js" type="text/javascript">
 </script>
 <script src="https://deo.shopeemobile.com/shopee/shopee-pcmall-live-sg/assets/bundle.73d2313c7936dfbf57b4.js" type="text/javascript">
 </script>
</body>



### Chrome F12 -> Network -> XHR

* 找到有item 的項目
![shopee_items](./shopee_items.png)

### 找到網址 `'https://shopee.tw/api/v2/search_items/?by=relevancy&keyword=cabinzero&limit=50&newest=0&order=desc&page_type=search&version=2'`
* 並用 get 取得
* header 會影響取得資料的差異(自行增減)
* if-none-match- 會影響資料取得的完整性
* `text = 'cabinzero'`
    * `'if-none-match-':'55b03-e02247ccaf743c03922273 bb6ee346e9'`
* 蝦皮好像有防爬蟲 多次抓取就需要 if-none-match-


In [5]:
# 轉換萬國碼
text = 'cabinzero'
# b = text.encode('utf-8')
# text = str(b)[2:-1].replace('\\x', '%').replace(' ', '%20')


url = f'https://shopee.tw/api/v2/search_items/?by=relevancy&keyword={text}&limit=50&newest=0&order=desc&page_type=search&version=2'

headers = {'referer':f'https://shopee.tw/search?keyword={text}',
           'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36',
           'if-none-match-':'55b03-e02247ccaf743c03922273bb6ee346e9'} # if-none-match- 這影響資料完整性？
r = requests.get(url, headers=headers)
shopee_items = r.json()['items']

In [6]:
for item in shopee_items:
    print(item['name'], '----->$', item['price']/1e5, item['currency'])

【加也】Cabinzero 軍規版 英國輕旅登機後背包、旅行包 36L ----->$ 2920.0 TWD
阿胖正品--現貨 CabinZero  登機包 大容量- 軍用特別款 (黑) 36L 44L 英國 輕旅 登機 後背包 ----->$ 2910.0 TWD
【加也】Cabinzero 軍規版 英國輕旅登機後背包 旅行包 44L ----->$ 3090.0 TWD
阿胖正品---現貨 CABINZERO 登機包 大容量- 軍用特別款 (沙漠色)- 36L 英國 輕旅 登機 後背包 ----->$ 2910.0 TWD
Cabin Zero 免Check-in 防水背囊 44L ----->$ 2499.0 TWD
【@newone】CabinZero 登機包 軍用特別款 英國 輕旅 流行 後背包 褐色 44L ----->$ 3298.0 TWD
毛仔嚴選 CabinZero 登機包-[大] 44L 英國輕旅登機後背包 大容量 10年保固 冰河灰 另有[小]跟[中] ----->$ 2580.0 TWD
毛仔嚴選 CabinZero 登機包-[大] 44L 英國輕旅登機後背包 大容量 10年保固 城市迷彩另有[小]跟[中] ----->$ 2580.0 TWD
Cabinzero全新防水包 附黑色收納包 ----->$ 1699.0 TWD
【CabinZero】英國輕旅登機後背包44L - 火鶴紅  只有一個售完不補 ----->$ 1880.0 TWD
Cabinzero 44L 登機包 ----->$ 2000.0 TWD
【CabinZero】英國輕旅登機後背包44L - 鯨魚藍 只有一個售完不補 ----->$ 1880.0 TWD
【加也】Cabinzero HIPPACK 腰包 2L ----->$ 830.0 TWD
【@newone】CabinZero 登機包 軍用特別款 英國 輕旅 流行 後背包 褐 深藍 軍綠 黑 36L ----->$ 3128.0 TWD
ST嚴選 CabinZero 登機包-[大.中.小] 英國輕旅登機後背包 大容量 10年保固  灰色 ----->$ 2580.0 TWD
S H C-🇬🇧英國 cabinzero 36L 軍用款限量特殊色 卡其 深藍 共兩色 旅行後背包 休閒背包 ----->$ 3680.0 

### 要完整抓取資料 必須獲得頁面的 `if-none-match-` 可是目前稿不懂產生機制

### 翻頁機制在newest=50*i (i 0~)  , 爬到第二頁就會跳錯誤

In [7]:
for i in range(3):
    url = f'https://shopee.tw/api/v2/search_items/?by=relevancy&keyword={text}&limit=50&newest={i*50}&order=desc&page_type=search&version=2'
    r = requests.get(url, headers=headers)
    shopee_items = r.json()['items']
    for item in shopee_items:
        print(item['name'], '----->$', item['price']/1e5, item['currency'])

【加也】Cabinzero 軍規版 英國輕旅登機後背包、旅行包 36L ----->$ 2920.0 TWD
阿胖正品--現貨 CabinZero  登機包 大容量- 軍用特別款 (黑) 36L 44L 英國 輕旅 登機 後背包 ----->$ 2910.0 TWD
【加也】Cabinzero 軍規版 英國輕旅登機後背包 旅行包 44L ----->$ 3090.0 TWD
阿胖正品---現貨 CABINZERO 登機包 大容量- 軍用特別款 (沙漠色)- 36L 英國 輕旅 登機 後背包 ----->$ 2910.0 TWD
Cabin Zero 免Check-in 防水背囊 44L ----->$ 2499.0 TWD
【@newone】CabinZero 登機包 軍用特別款 英國 輕旅 流行 後背包 褐色 44L ----->$ 3298.0 TWD
毛仔嚴選 CabinZero 登機包-[大] 44L 英國輕旅登機後背包 大容量 10年保固 冰河灰 另有[小]跟[中] ----->$ 2580.0 TWD
毛仔嚴選 CabinZero 登機包-[大] 44L 英國輕旅登機後背包 大容量 10年保固 城市迷彩另有[小]跟[中] ----->$ 2580.0 TWD
Cabinzero全新防水包 附黑色收納包 ----->$ 1699.0 TWD
【CabinZero】英國輕旅登機後背包44L - 火鶴紅  只有一個售完不補 ----->$ 1880.0 TWD
Cabinzero 44L 登機包 ----->$ 2000.0 TWD
【CabinZero】英國輕旅登機後背包44L - 鯨魚藍 只有一個售完不補 ----->$ 1880.0 TWD
【加也】Cabinzero HIPPACK 腰包 2L ----->$ 830.0 TWD
【@newone】CabinZero 登機包 軍用特別款 英國 輕旅 流行 後背包 褐 深藍 軍綠 黑 36L ----->$ 3128.0 TWD
ST嚴選 CabinZero 登機包-[大.中.小] 英國輕旅登機後背包 大容量 10年保固  灰色 ----->$ 2580.0 TWD
S H C-🇬🇧英國 cabinzero 36L 軍用款限量特殊色 卡其 深藍 共兩色 旅行後背包 休閒背包 ----->$ 3680.0 

## 使用 selenium (Chrome) 虛擬瀏覽器
* 獲取最後獲得的原始碼

In [8]:
# 打開瀏覽器(檔案位置)
chrome = webdriver.Chrome('/home/sppool/.local/share/chromedriver')

In [9]:
text = 'cabinzero'
url = f'https://shopee.tw/search?keyword={text}'
chrome.get(url)

# 網站需要讀取的時間
time.sleep(2)

### 只有15筆資料 (因為有些資料需要捲動到下面才會下載)

In [10]:
soup = bs4.BeautifulSoup(chrome.page_source, "html.parser")
n = 0
for tag in soup.select('a[data-sqe="link"]'):
    n += 1
    name = tag.select('div[data-sqe="name"]')[0].div.get_text()
    price = tag.select('span')[-1].get_text()
    print(f'{n} : {name} ----> ${price}')

1 : Cabin Zero 免Check-in 防水背囊 44L ----> $2,499
2 : 【@newone】CabinZero 登機包 軍用特別款 英國 輕旅 流行 後背包 褐 深藍 軍綠 黑 36L ----> $3,128
3 : 【加也】Cabinzero 軍規版 英國輕旅登機後背包 旅行包 44L ----> $3,090
4 : 【CabinZero】英國輕旅登機後背包44L - 鯨魚藍 只有一個售完不補 ----> $1,880
5 : 阿胖正品---現貨 CABINZERO 登機包 大容量- 軍用特別款 (沙漠色)- 36L 英國 輕旅 登機 後背包 ----> $2,910
6 : 阿胖正品---現貨 CabinZero 登機包 大容量- 軍用特別款 深藍 44L英國 輕旅 登機 後背包 ----> $3,080
7 : 阿胖正品--現貨 CabinZero  登機包 大容量- 軍用特別款 (黑) 36L 44L 英國 輕旅 登機 後背包 ----> $3,080
8 : Cabinzero 44L 登機包 ----> $2,000
9 : 【CabinZero】英國輕旅登機後背包44L - 火鶴紅  只有一個售完不補 ----> $1,880
10 : 【@newone】CabinZero 登機包 軍用特別款 英國 輕旅 流行 後背包 褐色 44L ----> $3,298
11 : 【加也】Cabinzero HIPPACK 腰包 2L ----> $830
12 : 毛仔嚴選 CabinZero 登機包-[大] 44L 英國輕旅登機後背包 大容量 10年保固 冰河灰 另有[小]跟[中] ----> $2,580
13 : Cabinzero全新防水包 附黑色收納包 ----> $1,699
14 : 毛仔嚴選 CabinZero 登機包-[大] 44L 英國輕旅登機後背包 大容量 10年保固 沙漠黃 另有[小]跟[中] ----> $2,580
15 : sonia_usa_korea-嚴選正品快速出貨 🇬🇧英國 cabin zero 登機包 後背包 旅行包 雙肩包 ----> $3,080


### [滾動視窗](https://selenium-python.readthedocs.io/faq.html#how-to-scroll-down-to-the-bottom-of-a-page)
* 瀏覽器要保持最大寬度, 有些連結才能順利打開
* javascript: `window.scrollTo(x, y)`

In [11]:
# 藉由滾動讀取所有的資料
for i in range(0, 4):
    chrome.execute_script(f"window.scrollTo(250, {i*1000});")
    time.sleep(1)

In [13]:
soup = bs4.BeautifulSoup(chrome.page_source, "html.parser")
n = 0
for tag in soup.select('a[data-sqe="link"]'):
    n += 1
    name = tag.select('div[data-sqe="name"]')[0].div.get_text()
    price = tag.select('span')[-1].get_text()
    print(f'{n} : {name} ----> ${price}')

1 : Cabin Zero 免Check-in 防水背囊 44L ----> $2,499
2 : 【@newone】CabinZero 登機包 軍用特別款 英國 輕旅 流行 後背包 褐 深藍 軍綠 黑 36L ----> $3,128
3 : 【加也】Cabinzero 軍規版 英國輕旅登機後背包 旅行包 44L ----> $3,090
4 : 【CabinZero】英國輕旅登機後背包44L - 鯨魚藍 只有一個售完不補 ----> $1,880
5 : 阿胖正品---現貨 CABINZERO 登機包 大容量- 軍用特別款 (沙漠色)- 36L 英國 輕旅 登機 後背包 ----> $2,910
6 : 阿胖正品---現貨 CabinZero 登機包 大容量- 軍用特別款 深藍 44L英國 輕旅 登機 後背包 ----> $3,080
7 : 阿胖正品--現貨 CabinZero  登機包 大容量- 軍用特別款 (黑) 36L 44L 英國 輕旅 登機 後背包 ----> $3,080
8 : Cabinzero 44L 登機包 ----> $2,000
9 : 【CabinZero】英國輕旅登機後背包44L - 火鶴紅  只有一個售完不補 ----> $1,880
10 : 【@newone】CabinZero 登機包 軍用特別款 英國 輕旅 流行 後背包 褐色 44L ----> $3,298
11 : 【加也】Cabinzero HIPPACK 腰包 2L ----> $830
12 : 毛仔嚴選 CabinZero 登機包-[大] 44L 英國輕旅登機後背包 大容量 10年保固 冰河灰 另有[小]跟[中] ----> $2,580
13 : Cabinzero全新防水包 附黑色收納包 ----> $1,699
14 : 毛仔嚴選 CabinZero 登機包-[大] 44L 英國輕旅登機後背包 大容量 10年保固 沙漠黃 另有[小]跟[中] ----> $2,580
15 : sonia_usa_korea-嚴選正品快速出貨 🇬🇧英國 cabin zero 登機包 後背包 旅行包 雙肩包 ----> $3,080
16 : 【@newone】CabinZero 登機包 英國 輕旅 流行 後背包 大 中 小 紅色 ----> $2,618
17 : 【@n

In [14]:
chrome.quit()